<a href="https://colab.research.google.com/github/monolixd/YoloV8-MoneyTransferSlip/blob/main/YoloV8MOneyTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())  # เช็คว่า CUDA ใช้งานได้หรือไม่
print("CUDA Version:", torch.version.cuda)  # เช็คเวอร์ชัน CUDA ที่ติดตั้ง
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")  # เช็คชื่อการ์ดจอที่ใช้

CUDA Available: True
CUDA Version: 12.1
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


# YOLOV8 Money Transfer Slip

## 1. ติดตั้ง Libraries ที่จำเป็น
ต้องติดตั้ง Libraries ที่จำเป็นสำหรับการทำงานกับ YOLOv8 และจัดการไฟล์ ZIP

In [ ]:
!pip install ultralytics

## 2. โหลดไฟล์ ZIP ขึ้นไปใน JupyterLab
คุณสามารถอัปโหลดไฟล์ ZIP ไปยัง JupyterLab ได้โดยตรง หรือใช้โค้ดเพื่อโหลดไฟล์จาก Google Drive, URL, หรือที่เก็บข้อมูลอื่นๆ

ตัวอย่างการอัปโหลดไฟล์ ZIP ไปยัง JupyterLab:

-ลากไฟล์ ZIP ไปวางในโฟลเดอร์ของ JupyterLab

## 3. แยกไฟล์ ZIP
หลังจากอัปโหลดไฟล์ ZIP แล้ว คุณต้องแยกไฟล์ออกมาเพื่อใช้งาน

In [ ]:
import zipfile
import os

# กำหนด path ไปยังไฟล์ ZIP
zip_path = "C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyNew.v1i.yolov8"

# สร้างโฟลเดอร์เพื่อเก็บไฟล์ที่แยก
extract_folder = "MoneyHeist-dataset"
os.makedirs(extract_folder, exist_ok=True)

# แยกไฟล์ ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("ไฟล์ถูกแยกไว้ที่:", extract_folder)

## 4. ตรวจสอบโครงสร้างโฟลเดอร์
### 4.1 โครงสร้างโฟลเดอร์ที่ถูกต้อง
YOLOv8 ต้องการโครงสร้างโฟลเดอร์ข้อมูลในรูปแบบต่อไปนี้:

### 4.2 ตรวจสอบโครงสร้างโฟลเดอร์
คุณสามารถใช้โค้ดต่อไปนี้เพื่อตรวจสอบว่าโฟลเดอร์มีโครงสร้างที่ถูกต้องหรือไม่:

In [ ]:
import os

# ตรวจสอบโครงสร้างโฟลเดอร์
def check_folder_structure(base_folder):
    required_folders = ['train', 'valid', 'test']
    required_subfolders = ['images', 'labels']

    for folder in required_folders:
        folder_path = os.path.join(base_folder, folder)
        if not os.path.exists(folder_path):
            print(f"โฟลเดอร์ {folder} ไม่พบ!")
            return False

        for subfolder in required_subfolders:
            subfolder_path = os.path.join(folder_path, subfolder)
            if not os.path.exists(subfolder_path):
                print(f"โฟลเดอร์ {subfolder} ใน {folder} ไม่พบ!")
                return False

    print("โครงสร้างโฟลเดอร์ถูกต้อง!")
    return True

# เรียกใช้ฟังก์ชันตรวจสอบ
base_folder = "MoneyHeist-dataset"
check_folder_structure(base_folder)

โครงสร้างโฟลเดอร์ถูกต้อง!


True

## 5. ตั้งค่า Configuration สำหรับ YOLOv8
หลังจากตรวจสอบโครงสร้างโฟลเดอร์แล้ว คุณต้องสร้างไฟล์ data.yaml เพื่อบอก YOLOv8 ว่าข้อมูลอยู่ที่ไหน และมีกี่คลาส

In [ ]:
import os

# กำหนด path ไปยังโฟลเดอร์ข้อมูล
base_folder = "C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset"

# จำนวนคลาสและชื่อคลาส (ปรับตามข้อมูลของคุณ)
num_classes = 2  # จำนวนคลาส
class_names = ['Fake_Count', 'Real_Count']  # ชื่อคลาส

# สร้างเนื้อหาไฟล์ data.yaml
config_content = f"""
train: {base_folder}/train/images
val: {base_folder}/valid/images
test: {base_folder}/test/images

nc: {num_classes}
names: {class_names}
"""

# ตรวจสอบว่าโฟลเดอร์ปลายทางมีอยู่หรือไม่ ถ้าไม่มีให้สร้างใหม่
os.makedirs(base_folder, exist_ok=True)

# กำหนด path ของไฟล์
yaml_path = os.path.join(base_folder, "data.yaml")

# เขียนไฟล์ data.yaml
with open(yaml_path, "w") as f:
    f.write(config_content)

# ตรวจสอบว่าไฟล์ถูกสร้างขึ้นสำเร็จหรือไม่
if os.path.exists(yaml_path):
    print(f"ไฟล์ data.yaml ถูกสร้างเรียบร้อยแล้ว! ที่ {yaml_path}")
else:
    print("เกิดข้อผิดพลาดในการสร้างไฟล์ data.yaml")


ไฟล์ data.yaml ถูกสร้างเรียบร้อยแล้ว! ที่ C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset\data.yaml


### 5.1 ตรวจสอบไฟล์ data.yaml
หลังจากสร้างไฟล์ data.yaml แล้ว คุณสามารถเปิดไฟล์นี้เพื่อตรวจสอบว่าข้อมูลถูกต้องหรือไม่ โดยใช้โค้ดต่อไปนี้:

In [ ]:
# ตรวจสอบไฟล์ data.yaml
with open(f"{base_folder}/data.yaml", "r") as f:
    print(f.read())

In [ ]:
import os
print(os.path.exists("MoneyHeist-dataset/train/images"))
print(os.path.exists("MoneyHeist-dataset/valid/images"))
print(os.path.exists("MoneyHeist-dataset/test/images"))

True
True
True


## 6. เทรนโมเดล YOLOv8
หลังจากตั้งค่า data.yaml แล้ว คุณสามารถเริ่มเทรนโมเดล YOLOv8 ได้เลย

In [ ]:
import os

paths = [
    "C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/train/images",
    "C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/valid/images",
    "C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/test/images",
]

for path in paths:
    print(f"Checking: {path} -> Exists: {os.path.exists(path)}")

Checking: C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/train/images -> Exists: True
Checking: C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/valid/images -> Exists: True
Checking: C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/test/images -> Exists: True


In [ ]:
from ultralytics import YOLO

# โหลดโมเดล
model = YOLO("yolov8n.pt")

# เทรนโมเดล
results = model.train(
    data="C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/data.yaml",  # ใช้ absolute path
    epochs=100,
    imgsz=640,
    batch=16,
    name="yolov8_custom_train"
)

New https://pypi.org/project/ultralytics/8.3.73 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.72  Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/realm/Desktop/YoloV8_MoneyVer2/MoneyHeist-dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_custom_train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_n

train: Scanning C:\Users\realm\Desktop\YoloV8_MoneyVer2\MoneyHeist-dataset\train\labels.cache... 1992 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1992/1992 [00:00<?, ?it/s]
val: Scanning C:\Users\realm\Desktop\YoloV8_MoneyVer2\MoneyHeist-dataset\valid\labels.cache... 249 images, 0 backgrounds, 0 corrupt: 100%|██████████| 249/249 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolov8_custom_train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\yolov8_custom_train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.16G      1.411      3.199      1.188          8        640: 100%|██████████| 125/125 [00:29<00:00,  4.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.14it/s]

                   all        249        249      0.563       0.53      0.518      0.334


...

    100/100      2.16G     0.7893     0.3717     0.9793          8        640: 100%|██████████| 125/125 [00:26<00:00,  4.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.65it/s]

                   all        249        249      0.995      0.995      0.995      0.805



100 epochs completed in 0.841 hours.
Optimizer stripped from runs\detect\yolov8_custom_train4\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\yolov8_custom_train4\weights\best.pt, 6.3MB

Validating runs\detect\yolov8_custom_train4\weights\best.pt...
Ultralytics 8.3.72  Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.42it/s]


                   all        249        249      0.994      0.993      0.995      0.807
            Fake_Count        115        115      0.995      0.991      0.995      0.798
            Real_Count        134        134      0.993      0.995      0.994      0.816
Speed: 0.6ms preprocess, 2.9ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs\detect\yolov8_custom_train4
